In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scanpy as sc
import os
import cv2

from micron2.codexutils import ImageMontage, layout_cells

from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
def read_color_config(fname):
    df = pd.read_csv(fname, index_col=None, header=None)
    df.columns = ['channel', 'color', 'low', 'high']
    colors = {}
    saturations = {}
    for i in range(df.shape[0]):
        colors[df.loc[i,'channel']] = df.loc[i,'color']
        saturations[df.loc[i,'channel']] = (df.loc[i,'low'], df.loc[i,'high'])
    return colors, saturations

def filter_coordinates(coords, s):
    valid_coords = np.ones(coords.shape[0], dtype=np.bool)
    max_1, max_2 = np.max(coords, axis=0)
    i = coords[:,0]<(s+1)
    i += coords[:,1]<(s+1)
    i += coords[:,0]>(max_1-s)
    i += coords[:,1]>(max_2-s)
    valid_coords[i] = 0
    return valid_coords

In [4]:
toss_regions = ['TMA3_reg1', 'TMA3_reg2', 'TMA3_reg7', 'TMA3_reg21', 'TMA3_reg23', 'TMA3_reg25', 'TMA3_reg33']
toss_regions += ['TMA2_reg2', 'TMA2_reg20', 'TMA2_reg26', 'TMA2_reg34']
def maybe_toss(s):
    for t in toss_regions:
        if t in s:
            return True
    return False
all_samples = !ls -d /storage/codex/preprocessed_data/*Bladder_TMA*
all_samples = [os.path.basename(s) for s in all_samples]
all_samples = [s for s in all_samples if 'reg0' not in s]

# all_samples = [f'210226_Bladder_TMA1_reg{i}' for i in range(1,38)]
all_samples = [f'210308_Bladder_TMA2_reg{i}' for i in range(1,37)]
# all_samples = [f'210311_Bladder_TMA3_reg{i}' for i in range(1,36)]
# all_samples = [s for s in all_samples if not maybe_toss(s)]
all_samples

['210308_Bladder_TMA2_reg1',
 '210308_Bladder_TMA2_reg2',
 '210308_Bladder_TMA2_reg3',
 '210308_Bladder_TMA2_reg4',
 '210308_Bladder_TMA2_reg5',
 '210308_Bladder_TMA2_reg6',
 '210308_Bladder_TMA2_reg7',
 '210308_Bladder_TMA2_reg8',
 '210308_Bladder_TMA2_reg9',
 '210308_Bladder_TMA2_reg10',
 '210308_Bladder_TMA2_reg11',
 '210308_Bladder_TMA2_reg12',
 '210308_Bladder_TMA2_reg13',
 '210308_Bladder_TMA2_reg14',
 '210308_Bladder_TMA2_reg15',
 '210308_Bladder_TMA2_reg16',
 '210308_Bladder_TMA2_reg17',
 '210308_Bladder_TMA2_reg18',
 '210308_Bladder_TMA2_reg19',
 '210308_Bladder_TMA2_reg20',
 '210308_Bladder_TMA2_reg21',
 '210308_Bladder_TMA2_reg22',
 '210308_Bladder_TMA2_reg23',
 '210308_Bladder_TMA2_reg24',
 '210308_Bladder_TMA2_reg25',
 '210308_Bladder_TMA2_reg26',
 '210308_Bladder_TMA2_reg27',
 '210308_Bladder_TMA2_reg28',
 '210308_Bladder_TMA2_reg29',
 '210308_Bladder_TMA2_reg30',
 '210308_Bladder_TMA2_reg31',
 '210308_Bladder_TMA2_reg32',
 '210308_Bladder_TMA2_reg33',
 '210308_Bladder_TM

# whole regions

In [28]:
data_home = '/storage/codex/preprocessed_data'
outdir = f'/storage/tmp-outgoing/codex-cell-images/bladder/epithelial_niche_examples'
outfile = 'Bladder_TMA3_PD-1.png'

all_samples = [f'210226_Bladder_TMA1_reg{i}' for i in range(1,38)]
# all_samples = [f'210308_Bladder_TMA2_reg{i}' for i in range(1,37)]
# all_samples = [f'210311_Bladder_TMA3_reg{i}' for i in range(1,36)]

all_samples += [f'210308_Bladder_TMA2_reg{i}' for i in range(1,37)]
all_samples += [f'210311_Bladder_TMA3_reg{i}' for i in range(1,36)]

ncol = 7
downsample = 0.5

if not os.path.exists(outdir):
    os.makedirs(outdir)

"""
['DAPI', 'aSMA', 'CD45', 'PDGFRb', 'CD49a', 'CD68', 'CD31', 'CD103',
 'HLA-DR', 'UPK3', 'GATA3', 'CD3e', 'Ki-67', 'CDH18', 'CDH12',
 'KRT13', 'KRT17', 'CK5-6', 'CDH1', 'PD-1', 'CD11c', 'KRT20',
 'CD69', 'PD-L2', 'CD20', 'PD-L1', 'FOXP3', 'ERBB2', 'GZMB', 'CD44',
 'CD45RO', 'LAG3', 'CD45RA', 'CD8', 'CD4', 'PanCytoK']
""" 
    

colors, saturations = read_color_config('/home/ingn/devel/micron2/projects/bladder/color_config.csv')
channel_groups = [
#     ['DAPI', 'CD4', 'CD3e', 'CD8', 'CD20', 'CD68'], # Immune
#     ['DAPI', 'CK5-6', 'KRT13', 'KRT17', 'CDH12', 'CDH18'] # Epithelial
#     ['DAPI', 'aSMA', 'PDGFRb', 'CD31', 'CDH1', 'PanCytoK'] # Stromal
#     ['DAPI', 'PanCytoK', 'CD45', 'CD31', 'aSMA'], # TME - 1
#     ['DAPI', 'CK5-6', 'KRT17', 'KRT13', 'UPK3', 'KRT20']
#     ['DAPI', 'PD-1', 'CD3e', 'CD8', 'CD49a'] # Tcell
#     ['DAPI', 'PD-1']
#     ['DAPI', 'GZMB', 'CD8', 'PD-1']
#     ['DAPI', 'CD68', 'CD8', 'CD3e', 'PD-1']
    ['DAPI', 'KRT13', 'CDH12', 'CD3e']
]

bbox = [50, 3800, 50, 3800]
dx = bbox[1]-bbox[0]

sample_images = []

for sample in all_samples:
    adataf = f'{data_home}/{sample}/{sample}.h5ad'
    outf = f'{outdir}/{sample}.png'
    
    import glob
    def get_source(image_home, ch):
        lst = glob.glob(f'{image_home}/*.tif')
        # srch = f'_{ch}_' if ch != 'DAPI' else f'_{ch}-011'
        srch = f'_{ch}_' if ch != 'DAPI' else f'_{ch}1_'
        for l in lst:
            if srch in l:
                return l

    sources = {ch: get_source(f'{data_home}/{sample}/images', ch) for ch in colors.keys()}
    montager = ImageMontage(sources=sources, colors=colors, saturations=saturations,
                            channel_groups=channel_groups)
    
    m = montager.montage(bbox)[0]
    m = cv2.resize(m, dsize=(0,0), fx=downsample, fy=downsample)
    m = montager.add_color_legend(m, 0, label_space=150, fontScale=2)
    
    print(f'{m.shape} --> {outf}')
    cv2.imwrite(outf, m[:,:,::-1])
    sample_images.append(m.copy())

##### FOR BLADDER insert 4 blanks up front

# # TMA1
# sample_images = [np.zeros(m.shape,dtype=np.uint8)]*4 + sample_images
# sample_images.insert(-6,np.zeros(m.shape,dtype=np.uint8))

# # TMA2
# sample_images = [np.zeros(m.shape,dtype=np.uint8)]*4 + sample_images
# sample_images.insert(7, np.zeros(m.shape,dtype=np.uint8))
# sample_images.insert(-6,np.zeros(m.shape,dtype=np.uint8))

# # TMA3
# sample_images = [np.zeros(m.shape,dtype=np.uint8)]*4 + sample_images
# sample_images.insert(7, np.zeros(m.shape,dtype=np.uint8))
# sample_images.insert(17, np.zeros(m.shape,dtype=np.uint8))
# sample_images.insert(-6,np.zeros(m.shape,dtype=np.uint8))


# out = layout_cells([sample_images], 0, ncol=ncol)
# cv2.imwrite(f'{outdir}/{outfile}', out[:,:,::-1])

(1939, 1875, 3) --> /storage/tmp-outgoing/codex-cell-images/bladder/epithelial_niche_examples/210226_Bladder_TMA1_reg1.png
(1939, 1875, 3) --> /storage/tmp-outgoing/codex-cell-images/bladder/epithelial_niche_examples/210226_Bladder_TMA1_reg2.png
(1939, 1875, 3) --> /storage/tmp-outgoing/codex-cell-images/bladder/epithelial_niche_examples/210226_Bladder_TMA1_reg3.png
(1939, 1875, 3) --> /storage/tmp-outgoing/codex-cell-images/bladder/epithelial_niche_examples/210226_Bladder_TMA1_reg4.png
(1939, 1875, 3) --> /storage/tmp-outgoing/codex-cell-images/bladder/epithelial_niche_examples/210226_Bladder_TMA1_reg5.png
(1939, 1875, 3) --> /storage/tmp-outgoing/codex-cell-images/bladder/epithelial_niche_examples/210226_Bladder_TMA1_reg6.png
(1939, 1875, 3) --> /storage/tmp-outgoing/codex-cell-images/bladder/epithelial_niche_examples/210226_Bladder_TMA1_reg7.png
(1939, 1875, 3) --> /storage/tmp-outgoing/codex-cell-images/bladder/epithelial_niche_examples/210226_Bladder_TMA1_reg8.png
(1939, 1875, 3) 

## all channels in a dataset at the same location

In [59]:
#
size = 100
# Trm's next to CDH12+ cells
# sample = f'210311_Bladder_TMA3_reg16'
# bbox = [743, 743+size, 3246, 3246+size]
# sample = f'210311_Bladder_TMA3_reg16'
# bbox = [740, 740+size, 3266, 3266+size]
# sample = f'210311_Bladder_TMA3_reg12'
# bbox = [2898, 2898+size, 1575, 1575+size]
# sample = f'210311_Bladder_TMA3_reg12'
# bbox = [1530, 1530+size, 2530, 2530+size]
 
## specific epithelial celltypes 
# sample = f'210311_Bladder_TMA3_reg35'
# bbox = [1310, 1310+size, 616, 616+size]
# sample = f'210226_Bladder_TMA1_reg15'
# bbox = [2629, 2629+size, 1201, 1201+size]
# sample = f'210311_Bladder_TMA3_reg12'
# bbox = [3140, 3140+size, 1000, 1000+size]
# sample = f'210226_Bladder_TMA1_reg14'
# # bbox = [641, 641+size, 2235, 2235+size]
# bbox = [633, 633+size, 1465, 1465+size]

## PD-1 Tcells next to CDH12+ cells
channel_groups = [['DAPI'], ['CDH12'], ['CD8'], ['PD-1']]
# sample = f'210311_Bladder_TMA3_reg24'
# bbox = [719, 719+size, 780, 780+size]
# sample = f'210308_Bladder_TMA2_reg7'
# bbox = [2597, 2597+size, 3305, 3305+size]
# sample = f'210226_Bladder_TMA1_reg8'
# bbox = [922, 922+size, 3178, 3178+size]
sample = f'210226_Bladder_TMA1_reg20'
bbox = [1644, 1644+size, 1596, 1596+size]

import glob
def get_source(image_home, ch):
    lst = glob.glob(f'{image_home}/*.tif')
    # srch = f'_{ch}_' if ch != 'DAPI' else f'_{ch}-011'
    srch = f'_{ch}_' if ch != 'DAPI' else f'_{ch}1_'
    for l in lst:
        if srch in l:
            return l
        
data_home = '/storage/codex/preprocessed_data'
outdir = f'/storage/tmp-outgoing/codex-cell-images/bladder/PD1_near_CDH12'
ncol = 2
# bbox = [100, 3800, 100, 3800]
bb = '_'.join([f'{b}' for b in bbox])

colors, saturations = read_color_config('color_config.csv')
# colors, saturations = read_color_config('color_config_cycles.csv')
# channel_groups = [[p] for p in colors.keys()]
# channel_groups = [['DAPI'], ['CDH12'], ['CD8'], ['CD49a']]
# channel_groups = [['DAPI'], ['CDH12'], ['KRT13'], ['KRT17']]
print(channel_groups)

# for r in range(1,37):
# sample = f'210311_Bladder_TMA3_reg{r}'

if not os.path.exists(outdir):
    os.makedirs(outdir)

outf = f'{outdir}/{sample}_{bb}.png'

sources = {ch: get_source(f'{data_home}/{sample}/images', ch) for ch in colors.keys()}
# F = False
# for s,v in sources.items():
#     if v is None:
#         print(f'{sample} {s}: {v}')
#         F = True
# if F:
#     continue
montager = ImageMontage(sources=sources, colors=colors, saturations=saturations,
                        channel_groups=channel_groups, verbose=False)

m = montager.montage(bbox, downsample=1)
legend_args = dict( fontScale=0.5, loc='corner', thickness=1, print_saturation=False)
# m = [montager.add_color_legend(m_, i, **legend_args) for i,m_ in enumerate(m)]

out = layout_cells([m], 0, padding=1, ncol=ncol, snake=False)
# out_legend = montager.add_color_legend(out, 0, label_space=175, fontScale=2)

print(f'{out.shape} --> {outf}')
cv2.imwrite(outf, out[:,:,::-1])

[['DAPI'], ['CDH12'], ['CD8'], ['PD-1']]
(200, 200, 3) --> /storage/tmp-outgoing/codex-cell-images/bladder/PD1_near_CDH12/210226_Bladder_TMA1_reg20_1644_1744_1596_1696.png


True

## Specific marker set at a certain location in one sample

In [58]:
# 
import glob
def get_source(image_home, ch):
    lst = glob.glob(f'{image_home}/*.tif')
    # srch = f'_{ch}_' if ch != 'DAPI' else f'_{ch}-011'
    srch = f'_{ch}_' if ch != 'DAPI' else f'_{ch}1_'
    for l in lst:
        if srch in l:
            return l
        
data_home = '/storage/codex/preprocessed_data'
outdir = f'/storage/tmp-outgoing/codex-cell-images/bladder/PD1_near_CDH12'
ncol = 6
if not os.path.exists(outdir):
    os.makedirs(outdir)

size = 100

## each epithelial cell type
# sample = f'210311_Bladder_TMA3_reg35'
# bbox = [1310, 1310+size, 616, 616+size]
# channel_groups = [['DAPI', 'KRT13', 'KRT17', 'CDH12']]

# sample = f'210226_Bladder_TMA1_reg15'
# bbox = [2629, 2629+size, 1201, 1201+size]
# channel_groups = [['DAPI', 'KRT13', 'KRT17', 'CDH12']]

# sample = f'210311_Bladder_TMA3_reg12'
# bbox = [3140, 3140+size, 1000, 1000+size]
# channel_groups = [['DAPI', 'KRT13', 'KRT17', 'CDH12']]

# sample = f'210226_Bladder_TMA1_reg14'
# bbox = [633, 633+size, 1465, 1465+size]
# channel_groups = [['DAPI', 'KRT13', 'KRT17', 'CDH12']]

# Trm's next to CDH12+ cells
# channel_groups = [['DAPI', 'CDH12', 'CD8', 'CD49a']]
# sample = f'210311_Bladder_TMA3_reg16'
# bbox = [740, 740+size, 3266, 3266+size]
# sample = f'210311_Bladder_TMA3_reg12'
# bbox = [2898, 2898+size, 1575, 1575+size]
# sample = f'210311_Bladder_TMA3_reg12'
# bbox = [1530, 1530+size, 2530, 2530+size]

## PD-1 Tcells next to CDH12+ cells
channel_groups = [['DAPI', 'CDH12', 'CD8', 'PD-1']]
# sample = f'210311_Bladder_TMA3_reg24'
# bbox = [719, 719+size, 780, 780+size]
# sample = f'210308_Bladder_TMA2_reg7'
# bbox = [2597, 2597+size, 3305, 3305+size]
# sample = f'210226_Bladder_TMA1_reg8'
# bbox = [922, 922+size, 3178, 3178+size]
sample = f'210226_Bladder_TMA1_reg20'
bbox = [1644, 1644+size, 1596, 1596+size]


bb = '_'.join([f'{b}' for b in bbox])

colors, saturations = read_color_config('color_config.csv')
print(channel_groups)

chs = '_'.join(channel_groups[0])
outf = f'{outdir}/{sample}_{chs}_{bb}.png'

sources = {ch: get_source(f'{data_home}/{sample}/images', ch) for ch in colors.keys()}
montager = ImageMontage(sources=sources, colors=colors, saturations=saturations,
                        channel_groups=channel_groups, verbose=False)

m = montager.montage(bbox, downsample=1)
legend_args = dict(fontScale=0.5, loc='legend', thickness=1, print_saturation=True, 
                   label_space=32, legend_ht=32, xstart=8)
# m = [montager.add_color_legend(m_, i, **legend_args) for i,m_ in enumerate(m)]

cv2.imwrite(outf, m[0][:,:,::-1])

[['DAPI', 'CDH12', 'CD8', 'PD-1']]


True